# Sport Vouchers Program Analysis

The goal of this Portfolio task is to explore data from the Federal Government Sport Vouchers program - this is a
program that provides up to two $100 vouchers for kids to participate in organised sport. Here's the [NSW Active Kids page](https://www.service.nsw.gov.au/transaction/apply-active-kids-voucher), there are similar schemes in other states - this data is from South Australia.

This is an exercise in exploring data and communicating the insights you can gain from it.  The source data comes
from the `data.gov.au` website and provides details of all Sport Vouchers that have been redeemed since February  in SA 2015 as part of the Sport Voucher program:  [Sports Vouchers Data](https://data.gov.au/dataset/ds-sa-14daba50-04ff-46c6-8468-9fa593b9f100/details).  This download is provided for you as `sportsvouchersclaimed.csv`.

To augment this data you can also make use of [ABS SEIFA data by LGA](http://stat.data.abs.gov.au/Index.aspx?DataSetCode=ABS_SEIFA_LGA#) which shows a few measures of Socioeconomic Advantage and Disadvantage for every Local Government Area. This data is provided for you as `ABS_SEIFA_LGA.csv`. This could enable you to answer questions about whether the voucher program is used equally by parents in low, middle and high socioeconomic areas.   You might be interested in this if you were concerned that this kind of program might just benifit parents who are already advantaged (they might already be paying for sport so this program wouldn't be helping much).

Questions:
* Describe the distribution of vouchers by: LGA, Sport - which regions/sports stand out? 
* Are some sports more popular in different parts of the state?
* Are any LGAs over/under represented in their use of vouchers?
* Is there a relationship between any of the SEIFA measures and voucher use in an LGA?

A challenge in this task is to display a useful summary of the data given that there are a large number of LGAs and sports involved.  Try to avoid long lists and large tables. Think about what plots and tables communicate the main points of your findings. 


# To-Do's
- create more summaries on columns
- create maps use geopandas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
%matplotlib inline

In [2]:
# read the sports vouchers data
sa_vouchers = pd.read_csv("files/sportsvouchersclaimed.csv")
sa_vouchers.head()

,Participant_ID,Participant_Suburb,Club_Previous_Member,Participant_ATSI_Indicator,Participant_CALD_Indicator,Participant_Sex,Voucher_Value,Club_Name,Voucher_Sport,Participant_LGA,Participant_Electorate,Voucher_Completion_Date,Voucher_Claim_Year
0,1000002,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Male,50,Flagstaff Hill Football Club,Australian Rules,ONKAPARINGA,FISHER,26/6/15 1:19,2015
1,1000002,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Male,50,Flagstaff Hill Football Club,Australian Rules,ONKAPARINGA,FISHER,29/6/16 23:55,2016
2,1000004,ABERFOYLE PARK,Not Registered Previously,Not ATSI,Not CALD,Female,50,Hub Gymnastics Club,Gymnastics,ONKAPARINGA,FISHER,21/4/16 3:37,2016
3,1000004,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Female,50,Hub Gymnastics Club,Gymnastics,ONKAPARINGA,FISHER,30/3/17 0:47,2017
4,1000005,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Female,50,Blackwood Community Recreation Association,Gymnastics,ONKAPARINGA,FISHER,20/7/15 2:50,2015


In [3]:
# check distribution of recurring redemption vs first joiners

In [4]:
# check distribution of aboriginal background vs non-aboriginal background

In [5]:
# check distribution of cultural heritage

In [6]:
# check distribution of genders

The SEIFA data includes row for each Local Government Area (LGA) but the names of the LGAs have a letter or letters in brackets after the name.  To allow us to match this up with the voucher data we remove this and convert to uppercase. 

For each LGA the data includes a number of measures all of which could be useful in your exploration.  

In [7]:
# read the SEIFA data, create an LGA column by removing the letters in brackets and converting to uppercase
seifa = pd.read_csv('files/ABS_SEIFA_LGA.csv')
lga = seifa["Local Government Areas - 2011"].str.replace(' \([ACSRCDMT]+\)', '', regex=True).str.upper()
seifa['LGA'] = lga
seifa.head()

,LGA_2011,Local Government Areas - 2011,INDEX_TYPE,Index Type,MEASURE,Measure,Value,LGA
0,10050,Albury (C),IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,967,ALBURY
1,10050,Albury (C),IRSAD,Index of Relative Socio-economic Advantage and...,MINS,Minimum score for SA1s in area,628,ALBURY
2,10050,Albury (C),IRSAD,Index of Relative Socio-economic Advantage and...,MAXS,Maximum score for SA1s in area,1169,ALBURY
3,10050,Albury (C),IRSAD,Index of Relative Socio-economic Advantage and...,URP,Usual resident population,47851,ALBURY
4,10110,Armidale Dumaresq (A),IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,985,ARMIDALE DUMARESQ


Since there are many rows per LGA we need to use `pivot_table` to create a new data frame with one row per LGA. Here
is an example of doing this to create a table with the different SCORE measures and the population (URP) field. 

In [8]:
LGA_scores = seifa[seifa.MEASURE == 'SCORE'].pivot_table(index="LGA", columns=["INDEX_TYPE"], values="Value")
LGA_scores.head()
LGA_pop = seifa[seifa.MEASURE == 'URP'].pivot_table(index="LGA", columns=["INDEX_TYPE"], values="Value")
LGA_scores['Population'] = LGA_pop.IEO
LGA_scores.head()

INDEX_TYPE,IEO,IER,IRSAD,IRSD,Population
LGA,,,,,
ADELAIDE,874.0,874.0,1036.0,1013.0,19606.0
ADELAIDE HILLS,1083.0,1083.0,1077.0,1081.0,38601.0
ALBANY,999.0,999.0,977.0,987.0,33614.0
ALBURY,964.0,964.0,967.0,979.0,47851.0
ALEXANDRINA,1000.0,1000.0,969.0,987.0,23693.0


This data frame can then be joined with the vouchers data fram to create one master data frame containing both the voucher data and the SEIFA measures.

In [9]:
sa_vouchers_scores = sa_vouchers.join(LGA_scores, on='Participant_LGA')
sa_vouchers_scores.head()

,Participant_ID,Participant_Suburb,Club_Previous_Member,Participant_ATSI_Indicator,Participant_CALD_Indicator,Participant_Sex,Voucher_Value,Club_Name,Voucher_Sport,Participant_LGA,Participant_Electorate,Voucher_Completion_Date,Voucher_Claim_Year,IEO,IER,IRSAD,IRSD,Population
0,1000002,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Male,50,Flagstaff Hill Football Club,Australian Rules,ONKAPARINGA,FISHER,26/6/15 1:19,2015,996.0,996.0,975.0,993.0,159517.0
1,1000002,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Male,50,Flagstaff Hill Football Club,Australian Rules,ONKAPARINGA,FISHER,29/6/16 23:55,2016,996.0,996.0,975.0,993.0,159517.0
2,1000004,ABERFOYLE PARK,Not Registered Previously,Not ATSI,Not CALD,Female,50,Hub Gymnastics Club,Gymnastics,ONKAPARINGA,FISHER,21/4/16 3:37,2016,996.0,996.0,975.0,993.0,159517.0
3,1000004,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Female,50,Hub Gymnastics Club,Gymnastics,ONKAPARINGA,FISHER,30/3/17 0:47,2017,996.0,996.0,975.0,993.0,159517.0
4,1000005,ABERFOYLE PARK,Registered Previously,Not ATSI,Not CALD,Female,50,Blackwood Community Recreation Association,Gymnastics,ONKAPARINGA,FISHER,20/7/15 2:50,2015,996.0,996.0,975.0,993.0,159517.0


In [10]:
sa_vouchers_scores[sa_vouchers_scores["Participant_LGA"].str.contains('ADELAIDE PLAINS') == True]

,Participant_ID,Participant_Suburb,Club_Previous_Member,Participant_ATSI_Indicator,Participant_CALD_Indicator,Participant_Sex,Voucher_Value,Club_Name,Voucher_Sport,Participant_LGA,Participant_Electorate,Voucher_Completion_Date,Voucher_Claim_Year,IEO,IER,IRSAD,IRSD,Population
6954,1006955,BARABBA,Not Registered Previously,Not ATSI,Not CALD,Male,50,S.A.N.F.L,Australian Rules,ADELAIDE PLAINS,GOYDER,30/3/17 0:46,2017,NaN,NaN,NaN,NaN,NaN
6955,1006956,BARABBA,Not Registered Previously,Not ATSI,Not CALD,Male,50,Mallala Football Club,Australian Rules,ADELAIDE PLAINS,GOYDER,14/9/15 5:20,2015,NaN,NaN,NaN,NaN,NaN
6956,1006957,BARABBA,Registered Previously,Not ATSI,Not CALD,Female,50,Mallala Netball Club,Netball,ADELAIDE PLAINS,GOYDER,30/3/17 0:37,2017,NaN,NaN,NaN,NaN,NaN
6957,1006960,BARABBA,Not Registered Previously,Not ATSI,Not CALD,Male,50,Hamley Bridge Football Club,Australian Rules,ADELAIDE PLAINS,GOYDER,28/8/15 2:28,2015,NaN,NaN,NaN,NaN,NaN
6958,1006960,BARABBA,Registered Previously,Not ATSI,Not CALD,Male,50,S.A.N.F.L,Australian Rules,ADELAIDE PLAINS,GOYDER,17/6/16 1:03,2016,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123703,1123704,WINDSOR,Not Registered Previously,Not ATSI,Not CALD,Male,50,S.A.N.F.L,Australian Rules,ADELAIDE PLAINS,GOYDER,17/6/16 1:03,2016,NaN,NaN,NaN,NaN,NaN
123704,1123705,WINDSOR,Registered Previously,Not ATSI,Not CALD,Female,50,Mallala Netball Club,Netball,ADELAIDE PLAINS,GOYDER,30/5/17 7:58,2017,NaN,NaN,NaN,NaN,NaN
123705,1123706,WINDSOR,Not Registered Previously,Not ATSI,Not CALD,Female,50,Long Plains Netball Club,Netball,ADELAIDE PLAINS,GOYDER,25/2/16 2:23,2015,NaN,NaN,NaN,NaN,NaN
123706,1123707,WINDSOR,Not Registered Previously,Not ATSI,Not CALD,Male,50,S.A.N.F.L,Australian Rules,ADELAIDE PLAINS,GOYDER,28/4/17 5:04,2017,NaN,NaN,NaN,NaN,NaN


### Description of voucher data:

- __Participant_ATSI_Indicator:__ Categorises the participant as identifying with aboriginal or Torres Strait Island heritage (ATSI), or not (Not ATSI)
- __Participant_CALD_Indicator:__ Categorises the participant as identifying with a non-Australian cultural background (CALD) or not (Not CALD) 
- __Voucher_Sport:__ Description of the activity undertaken using a sport voucher

### Description of seifa data:

__Low score indicates most disadvantaged.__

- __IRSAD:__ The Index of Relative Socio-economic Advantage and Disadvantage (IRSAD) summarises information about the economic and social conditions of people and households within an area, including both relative advantage and disadvantage measures.
- __IRSD:__ The Index of Relative Socio-economic Disadvantage (IRSD) is a general socio-economic index that summarises a range of information about the economic and social conditions of people and households within an area.
- __IER:__ The Index of Economic Resources (IER) focuses on the financial aspects of relative socio-economic advantage and disadvantage, by summarising variables related to income and wealth. This index excludes education and occupation variables because they are not direct measures of economic resources. It also misses some assets such as savings or equities which, although relevant, could not be included because this information was not collected in the 2011 Census.
- __IEO:__ The Index of Education and Occupation (IEO) is designed to reflect the educational and occupational level of communities.

### For later visualisation purposes, LGA map data will be read into the notebook

In [11]:
# get url of geojson file
nsw_url = "https://data.gov.au/geoserver/nsw-local-government-areas/wfs?request=GetFeature&typeName=ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e&outputFormat=json"

# read data in
nsw_lga_gdf = gpd.read_file(nsw_url)
nsw_lga_gdf.head()

,id,lg_ply_pid,dt_create,dt_retire,lga_pid,nsw_lga_sh,nsw_lga__1,nsw_lga__2,nsw_lga__3,nsw_lga__4,nsw_lga__5,geometry
0,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.1,649,2011-11-29,None,NSW153,2011-11-29,None,UNINCORPORATED,UNINCORPORATED,None,1,"MULTIPOLYGON (((159.10859 -31.54003, 159.10860..."
1,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.2,650,2011-11-29,None,NSW153,2011-11-29,None,UNINCORPORATED,UNINCORPORATED,None,1,"MULTIPOLYGON (((159.05968 -31.53428, 159.06000..."
2,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.3,651,2011-11-29,None,NSW153,2011-11-29,None,UNINCORPORATED,UNINCORPORATED,None,1,"MULTIPOLYGON (((159.07080 -31.49635, 159.07070..."
3,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.4,652,2011-11-29,None,NSW153,2011-11-29,None,UNINCORPORATED,UNINCORPORATED,None,1,"MULTIPOLYGON (((159.07386 -31.50180, 159.07380..."
4,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.5,653,2011-11-29,None,NSW153,2011-11-29,None,UNINCORPORATED,UNINCORPORATED,None,1,"MULTIPOLYGON (((159.07297 -31.59958, 159.07276..."


In [12]:
nsw_lga_gdf.tail()

,id,lg_ply_pid,dt_create,dt_retire,lga_pid,nsw_lga_sh,nsw_lga__1,nsw_lga__2,nsw_lga__3,nsw_lga__4,nsw_lga__5,geometry
192,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.193,712,2011-11-29,None,NSW306,2011-11-29,None,SHELLHARBOUR CITY COUNCIL,SHELLHARBOUR,None,1,"MULTIPOLYGON (((150.86547 -34.62580, 150.86546..."
193,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.194,643,2011-11-29,None,NSW310,2011-11-29,None,WOLLONGONG CITY COUNCIL,WOLLONGONG,None,1,"MULTIPOLYGON (((150.92662 -34.49192, 150.92662..."
194,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.195,644,2011-11-29,None,NSW310,2011-11-29,None,WOLLONGONG CITY COUNCIL,WOLLONGONG,None,1,"MULTIPOLYGON (((150.93647 -34.49425, 150.93647..."
195,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.196,645,2011-11-29,None,NSW310,2011-11-29,None,WOLLONGONG CITY COUNCIL,WOLLONGONG,None,1,"MULTIPOLYGON (((150.92312 -34.49002, 150.92312..."
196,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.197,647,2011-11-29,None,NSW153,2011-11-29,None,UNINCORPORATED,UNINCORPORATED,None,1,"MULTIPOLYGON (((159.25094 -31.75194, 159.25094..."


In [14]:
# drop duplicates and remove unnecessary columns
nsw_lga_gdf = nsw_lga_gdf.sort_values("dt_create",ascending=False).drop_duplicates(["lga_pid"])
nsw_lga_gdf = nsw_lga_gdf.drop(['lg_ply_pid', 'dt_create', 'dt_retire', 'nsw_lga_sh', 'nsw_lga__1', 'nsw_lga__2'],
                               axis=1)
nsw_lga_gdf.head()

,id,lga_pid,nsw_lga__3,nsw_lga__4,nsw_lga__5,geometry
118,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.119,NSW333,DUBBO REGIONAL,None,1,"MULTIPOLYGON (((148.62534 -31.90056, 148.62542..."
146,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.147,NSW153,UNINCORPORATED,None,1,"MULTIPOLYGON (((151.27359 -33.85086, 151.27358..."
176,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.177,NSW213,NORTH SYDNEY,None,1,"MULTIPOLYGON (((151.21456 -33.81584, 151.21461..."
151,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.152,NSW268,BURWOOD,None,1,"MULTIPOLYGON (((151.10096 -33.86843, 151.10106..."
150,ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e.151,NSW232,MOSMAN,None,1,"MULTIPOLYGON (((151.24728 -33.80618, 151.24729..."


In [15]:
# merge with sa_voucher data
map_df = pd.merge(nsw_lga_gdf, sa_vouchers_scores, how='right', left_on = 'nsw_lga__3', right_on = 'Participant_LGA')
map_df.head()

,id,lga_pid,nsw_lga__3,nsw_lga__4,nsw_lga__5,geometry,Participant_ID,Participant_Suburb,Club_Previous_Member,Participant_ATSI_Indicator,...,Voucher_Sport,Participant_LGA,Participant_Electorate,Voucher_Completion_Date,Voucher_Claim_Year,IEO,IER,IRSAD,IRSD,Population
0,NaN,NaN,NaN,NaN,NaN,None,1000002,ABERFOYLE PARK,Registered Previously,Not ATSI,...,Australian Rules,ONKAPARINGA,FISHER,26/6/15 1:19,2015,996.0,996.0,975.0,993.0,159517.0
1,NaN,NaN,NaN,NaN,NaN,None,1000002,ABERFOYLE PARK,Registered Previously,Not ATSI,...,Australian Rules,ONKAPARINGA,FISHER,29/6/16 23:55,2016,996.0,996.0,975.0,993.0,159517.0
2,NaN,NaN,NaN,NaN,NaN,None,1000004,ABERFOYLE PARK,Not Registered Previously,Not ATSI,...,Gymnastics,ONKAPARINGA,FISHER,21/4/16 3:37,2016,996.0,996.0,975.0,993.0,159517.0
3,NaN,NaN,NaN,NaN,NaN,None,1000004,ABERFOYLE PARK,Registered Previously,Not ATSI,...,Gymnastics,ONKAPARINGA,FISHER,30/3/17 0:47,2017,996.0,996.0,975.0,993.0,159517.0
4,NaN,NaN,NaN,NaN,NaN,None,1000005,ABERFOYLE PARK,Registered Previously,Not ATSI,...,Gymnastics,ONKAPARINGA,FISHER,20/7/15 2:50,2015,996.0,996.0,975.0,993.0,159517.0


In [16]:
# check for missing geometry 
map_df[map_df['geometry'] == 'None']

,id,lga_pid,nsw_lga__3,nsw_lga__4,nsw_lga__5,geometry,Participant_ID,Participant_Suburb,Club_Previous_Member,Participant_ATSI_Indicator,...,Voucher_Sport,Participant_LGA,Participant_Electorate,Voucher_Completion_Date,Voucher_Claim_Year,IEO,IER,IRSAD,IRSD,Population


In [21]:
# check for not matching values
nsw_lga_gdf[nsw_lga_gdf['nsw_lga__3'].str.contains('ONKAPARINGA', case=False)]

,id,lga_pid,nsw_lga__3,nsw_lga__4,nsw_lga__5,geometry


### 1. Describe the distribution of vouchers by: LGA, Sport - which regions/sports stand out?

#### 1.1 By LGA

In [ ]:
# group by LGA
LGA_vouchers = sa_vouchers_scores.groupby(
    "Participant_LGA")["Participant_LGA"].count().to_frame(name = 'Voucher_Count').reset_index()
LGA_vouchers

In [ ]:
# show distribution of vouchers by LGA
sns.set_theme(style="darkgrid")
plt.figure(figsize=(15, 18))
LGA_vouchers = LGA_vouchers.sort_values("Voucher_Count", ascending = False)
sns.barplot(data=LGA_vouchers, x="Voucher_Count", y="Participant_LGA")
plt.title("Distribution of Vouchers by LGA", fontsize=15)
plt.xlabel("Number of Vouchers", fontsize = 13)
plt.ylabel("");

In [ ]:
LGA_vouchers["Voucher_prop"] = round(LGA_vouchers["Voucher_Count"] / sum(LGA_vouchers["Voucher_Count"]) * 100, 2)
LGA_vouchers = LGA_vouchers.sort_values(by="Voucher_prop", ascending = False)
print("Proportion of number of vouchers for top 15 LGAs:\t", LGA_vouchers["Voucher_prop"].nlargest(15).sum())
print("Total number of LGAs:\t", LGA_vouchers.shape[0])

### Findings of LGA analysis:
It can clearly be seen that there is a highly uneven distribution between different areas. There are 69 LGAs in the dataset but only 15 of those account for around 64% of vouchers that have been redeemed throughout the state.

#### 1.2 By Sports

In [ ]:
# group by sports
sports_vouchers = sa_vouchers_scores.groupby(
    "Voucher_Sport")["Voucher_Sport"].count().to_frame(name = 'Voucher_Count').reset_index()
sports_vouchers = sports_vouchers.sort_values(by='Voucher_Count', ascending= False)
sports_vouchers.head(10)

In [ ]:
# calculate proportion of vouchers per sport, then show where 80% of vouchers are redeemed
sports_vouchers["Voucher_prop"] = round(sports_vouchers["Voucher_Count"]/sum(sports_vouchers["Voucher_Count"])*100, 2)
sports_vouchers.head(10)
    

In [ ]:
sports_vouchers["Voucher_prop"].head(10).sum(), sports_vouchers.shape

In [ ]:
# show distribution of vouchers by sports
plt.figure(figsize=(15, 18))
sns.barplot(data=sports_vouchers, x="Voucher_Count", y="Voucher_Sport")
plt.title("Distribution of Vouchers by Sport", fontsize=15)
plt.xlabel("Number of Vouchers", fontsize = 13)
plt.ylabel("");

In [ ]:
# show sports where less than 50 vouchers have been redeemed
sports_vouchers[sports_vouchers["Voucher_Count"] < 50]

In [ ]:
# show distribution of vouchers by more relevant sports
plt.figure(figsize=(15, 5))
sns.barplot(data=sports_vouchers.nlargest(10, "Voucher_Count"), x="Voucher_Count", y="Voucher_Sport")
plt.title("Distribution of Vouchers by Sport", fontsize=15)
plt.xlabel("Number of Vouchers", fontsize = 13)
plt.ylabel("");

### Findings of Sports analysis:

It can clearly be seen that the distribution of voucher redemption by sports is highly uneven. There are 50 sports to choose from, however, 10 of these account for over 90% of vouchers.

### 2. Are some sports more popular in different parts of the state?

In [ ]:
# select 10 most popular sports and 15 most populated states
LGA_top15 = LGA_vouchers.nlargest(15, "Voucher_Count")
sports_top10 = sports_vouchers.nlargest(10, "Voucher_Count")
sports_top10

In [ ]:
# create list with popular sport
sports_list = sports_top10["Voucher_Sport"].tolist()
sports_list

In [ ]:
# group by participants LGA and sports
LGA_sports_vouchers = sa_vouchers_scores.groupby(
    ["Participant_LGA", "Voucher_Sport"])["Participant_LGA"].count().to_frame(name = 'Voucher_Count').reset_index()
LGA_sports_vouchers

In [ ]:
# drop records that are not in list with most popular sport
LGA_sports_clean = LGA_sports_vouchers[LGA_sports_vouchers["Voucher_Sport"].isin(sports_list)]
LGA_sports_clean

In [ ]:
# reshape grouped output
LGA_sports_piv = pd.pivot_table(LGA_sports_clean, index="Participant_LGA", columns="Voucher_Sport", values="Voucher_Count")
LGA_sports_piv

In [ ]:
plt.figure(figsize=(13, 15))
sns.heatmap(LGA_sports_piv, vmin=0, vmax=5000, cmap=sns.cm.rocket_r, linewidths=0.1)
plt.title("Sport Voucher Distribution by LGA", fontsize = 15)
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=45);

There a few things to note about this heatmap:
1. Since all LGAs are included, there are a lot of empty values. Thus, it can be seen in which areas more or less vouchers are redeemed. In the following, another heatmap will be shown to get a more granular view between states that have higher voucher redemption. Actually, as it has been shown, this is due to population size and the hypothesis whether there is a difference between rural and metropolitan areas will be tested later with plotting a map.
2. The popluarity of sports is visualised on the map. The more popular a sport is, the less missing values it has across LGAs. It can be seen that vouchers for Australian Rules are redeemed in every single area, while Netball vouchers are redeemed in almost every area. This underlines their popularity throughout the state.

In [ ]:
# create list with 15 most populated areas
LGA_list = LGA_top15["Participant_LGA"].tolist()
LGA_list

In [ ]:
# drop records from grouped df that are not in list with most populated areas
LGA_sports_clean2 = LGA_sports_clean[LGA_sports_clean["Participant_LGA"].isin(LGA_list)]
LGA_sports_clean2

In [ ]:
# reshape grouped output
LGA_sports_piv2 = pd.pivot_table(LGA_sports_clean2, 
                                 index="Voucher_Sport", 
                                 columns="Participant_LGA", 
                                 values="Voucher_Count")
LGA_sports_piv2

In [ ]:
# create heatmap with most populated areas
plt.figure(figsize=(15, 10))
sns.heatmap(LGA_sports_piv2, vmin=0, vmax=5000, cmap=sns.cm.rocket_r, linewidth=0.1)
plt.title("Sport Voucher Distribution by LGA and Sports Type", fontsize = 15)
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=80);

Findings:


In [ ]:
# create map with geopandas to verify distinction between rural and metropolitan
# create new pivot tables 
map_pivot = pd.pivot_table(LGA_sports_vouchers, 
                           index="Participant_LGA", 
                           columns="Voucher_Sport", 
                           values="Voucher_Count")
map_pivot

In [ ]:
# geocode the LGAs

path = ""

LGA_geo = 

### 3. Are any LGAs over/under represented in their use of vouchers?

# calculate mean value of voucher amount and subtract from value to show over/under
# double check missing values

In [ ]:
# extract LGAs and respective population from main dataframe
population = sa_vouchers_scores.drop_duplicates("Participant_LGA")[["Participant_LGA", "Population"]]
population.sort_values(by='Participant_LGA')

In [ ]:
population[population["Participant_LGA"] == "ADELAIDE PLAINS"]

In [ ]:
# Concatenate to single dataframe
LGA_vouchers.sort_values(by='Participant_LGA')
pop_lga = pd.merge(population, LGA_vouchers, 'inner', 'Participant_LGA')
pop_lga

In [ ]:
pop_lga[pop_lga["Participant_LGA"] == "ADELAIDE PLAINS"]

In [ ]:
# show correlation between population and number of vouchers
pop_lga.corr()

Since there is a high correlation, this indicates that the number of vouchers per area should be considered relatively to population size for comparison. <br >
__Note:__ In the pop_lga dataframe it can be seen that there is a LGA with population size zero but 899 vouchers redeemed. Since this should not be the case, further analysis in this regard will be done in the following before showing the distribution of number of vouchers per LGA relative to population size.

In [ ]:
# check spelling of these LGAs in census data
seifa[seifa["LGA"].str.contains('ADELAIDE PLAINS') == True]

In [ ]:
# check spelling of these LGAs in voucher data
sa_vouchers[sa_vouchers["Participant_LGA"].str.contains('ADELAIDE PLAINS') == True]

In [ ]:
# remove records where values for population are missing
pop_lga_clean = pop_lga.dropna()
pop_lga_clean

In [ ]:
# create column of number of vouchers per thousand people in each LGA
pop_lga_clean["Voucher_Per_100"] = round(
    (pop_lga_clean["Voucher_Count"] / pop_lga_clean["Population"]) * 100, 2)
pop_lga_clean = pop_lga_clean.sort_values("Voucher_Per_100", ascending = False)
pop_lga_clean

In [ ]:
plt.figure(figsize=(15, 18))
sns.barplot(data=pop_lga_clean, x="Voucher_Per_100", y="Participant_LGA")
plt.title("Distribution of Vouchers by LGA per 100 people", fontsize=15)
plt.xlabel("Number of Vouchers", fontsize = 13)
plt.ylabel("");

### Findings of LGA analysis:

For this analysis, the few LGAs where no seifa data has been collected, have been omitted. For the remaining LGAs it can be seen that, first, the distribution of redeemed vouchers has changed by a lot and, second, the distribution is more even than without taking population size into consideration.<br >
For example, in Karoonda East Murray, only 388 vouchers were redeemed which puts them at the lower end of number of number of vouchers per LGA. However, after considering population size, it becomes clear that out of 100 people in this LGA, around 38 redeem sports vouchers. Through this analysis it has been shown where sports vouchers are more or less popular.

### 4. Is there a relationship between any of the SEIFA measures and voucher use in an LGA?

## Challenge - Queensland

_Note: this is an extra task that you might take on to get a better grade for your portfolio.  You can get a good pass grade without doing this._ 

Queensland has a similar program called [Get Started](https://data.gov.au/dataset/ds-qld-3118838a-d425-48fa-bfc9-bc615ddae44e/details?q=get%20started%20vouchers) and we can retrieve data from their program in a similar format.  

The file [round1-redeemed_get_started_vouchers.csv](files/round1-redeemed_get_started_vouchers.csv) contains records of the vouchers issued in Queensland. The date of this data is not included but the program started in 2015 so it is probably from around then.  

The data includes the LGA of the individual but the name of the activity is slightly different.  To do a comparable analysis you would need to map the activity names onto those from South Australia. 

In [ ]:
qld_vouchers = pd.read_csv('files/round1-redeemed_get_started_vouchers.csv')
qld_vouchers.head()

In [ ]:
# Join the QLD data with the LGA data as before to get population and SIEFA data integrated
qld_vouchers['LGA'] = qld_vouchers['Club Local Government Area Name'].str.replace(' \([RC]+\)', '').str.upper()
qld_vouchers_scores = qld_vouchers.join(LGA_scores, on='LGA')
qld_vouchers_scores.head()